# Predict Water Amount Needed

## Get and Prepare Weather Data from DataBase

In [1]:
import pymysql.cursors
import pandas as pd
import numpy as np

pd.options.display.max_rows = 200
#pd.set_option('display.float_format', lambda x: '%.20f' % x) #Display as Float
pd.set_option('display.float_format', lambda x: '{:,}'.format(x)) #Display as Scientific


connection = pymysql.connect(host = "mikmak.cc", user="sensor", passwd="Gaffe2017", db="weatherDW")
query = ('SELECT * FROM log_v_last24Hours WHERE S_Text <> "None"')

with connection.cursor() as cursor:
    cursor.execute(query)
connection.commit()
e_Log = cursor.fetchall()
connection.close()

e_Log = (np.array(e_Log))

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

t_Log = e_Log

t_Log = pd.DataFrame(data = t_Log[1:,:],
                     index = t_Log[1:,2],
                     columns =["ID","Value","TimeStamp"] )

t_Log = t_Log[t_Log != '-'] #DataSet contains some missing values, remove them

t_Log = t_Log.pivot(index='TimeStamp', columns='ID', values='Value')
t_Log = t_Log.sort_index(ascending = True) #It should be ascending, for rolling calculation later
t_Log = t_Log.dropna(how = "any")
t_Log = t_Log.astype(float)

t_Log.tail()


ID,Humidity,Pressure,Rain,Sunshine,Temperature,Wind
TimeStamp,,,,,,
2017-04-27 12:36:00,89.0,957.2,0.2,0.0,3.8,5.4
2017-04-27 13:36:01,91.0,957.0,0.1,0.0,3.7,7.2
2017-04-27 14:36:01,91.0,957.0,0.1,0.0,3.9,5.0
2017-04-27 15:36:08,87.0,957.0,0.0,0.0,3.8,4.7
2017-04-27 16:36:03,88.0,956.7,0.2,0.0,3.5,2.5


## Transform Data into Input Vector

### Order Data and Calculate Means and Sums

In [3]:
X = pd.DataFrame()

lastRow = len(t_Log.axes[0]) - 1

X = X.assign(tre200b0=t_Log.Temperature[[lastRow]])
X = X.assign(ure200b0=t_Log.Humidity[[lastRow]])
X = X.assign(rre200b0=t_Log.Rain[[lastRow]])
X = X.assign(sre000b0=t_Log.Sunshine[[lastRow]])
X = X.assign(fu3010b0=t_Log.Wind[[lastRow]])
X = X.assign(prestab0=t_Log.Pressure[[lastRow]])
#"tre200b0","ure200b0","rre150b0","sre000b0","fu3010b0","prestab0"

t_LogRolling = t_Log.rolling(len(t_Log.axes[0]))

X = X.assign(tre200b0_mean=t_LogRolling.Temperature.mean())
X = X.assign(ure200b0_mean=t_LogRolling.Humidity.mean())
X = X.assign(rre200b0_sum=t_LogRolling.Rain.sum())
X = X.assign(sre000b0_sum=t_LogRolling.Sunshine.sum())
X = X.assign(fu3010b0_mean=t_LogRolling.Wind.mean())
X = X.assign(prestab0_mean=t_LogRolling.Pressure.mean())

X.transpose()

TimeStamp,2017-04-27 16:36:03
tre200b0,3.5
ure200b0,88.0
rre200b0,0.2
sre000b0,0.0
fu3010b0,2.5
prestab0,956.7
tre200b0_mean,3.5956521739130443
ure200b0_mean,89.91304347826087
rre200b0_sum,1.5
sre000b0_sum,0.0


### Poly Features etc.

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.preprocessing import PolynomialFeatures

polyDegree = 5
poly = PolynomialFeatures(degree=polyDegree)
X = poly.fit_transform(X).astype(int)

#poly.n_output_features_ 
#pd.DataFrame(X).transpose()

/usr/local/lib/python3.4/dist-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


## Classify!
Use Logistic Regression Modell to decide if 
- no water at all or 
- a lot of water or 
- normal amount of water
is needed

### Load Model

In [5]:
from sklearn.externals import joblib
model = joblib.load('data/logreg_5deg.pkl')

### Do Calculation and Show Result
- 0 - means a lot of water
- 1 - is medium
- 2 - device should stay off

In [6]:
result = model.predict(X)[0]
result

from sklearn import preprocessing
le = joblib.load('data/leModel.pkl')
le.inverse_transform(result)


1

'Med'

This is very good!

In [7]:
#pd.DataFrame(model.coef_.transpose())

In [8]:
if result == 1:
    model_reg = joblib.load('data/linreg_med_5deg.pkl')
    res = model_reg.predict(X)[0]
else:
    model_reg = joblib.load('data/linreg_med_5deg.pkl')
    res = model_reg.predict(X)[0]

    
res

350.48208498691338

In [9]:

K = 10
np.argpartition(model_reg.coef_,-K)[-K:]

array([3980, 4176, 4171, 4173, 3998, 4052, 4050, 5748, 5730, 6167])